In [1]:
%matplotlib inline
import numpy as np
from tqdm import tqdm, tqdm_notebook
from os import makedirs
from os.path import exists, join
from shutil import copyfile
from skimage import io, transform

In [2]:
x_train_paths = np.load("../Cache/LPD/original/x_train.npy")
x_test_paths = np.load("../Cache/LPD/original/x_test.npy")
x_validation_paths = np.load("../Cache/LPD/original/x_validation.npy")
y_train_paths = np.load("../Cache/LPD/original/y_train.npy")
y_test_paths = np.load("../Cache/LPD/original/y_test.npy")
y_validation_paths = np.load("../Cache/LPD/original/y_validation.npy")

In [3]:
print(x_train_paths[0], y_train_paths[0])
print(x_train_paths[0].split("/")[-1][:-4])
im = io.imread(x_train_paths[0])
print(im.shape)
print(type(y_train_paths[0][0]))

../UFPR-ALPR-dataset/training/track0008/track0008[18].png [115 119  25  77 743 307 285 245]
track0008[18]
(1080, 1920, 3)
<class 'numpy.int64'>


In [4]:
print(x_train_paths.shape, y_train_paths.shape)
X = np.concatenate((x_train_paths, x_test_paths, x_validation_paths))
Y = np.concatenate((y_train_paths, y_test_paths, y_validation_paths))
print(X.shape, Y.shape)
print(x_test_paths[0], X[1800])

(1800,) (1800, 8)
(4500,) (4500, 8)
../UFPR-ALPR-dataset/testing/track0109/track0109[10].png ../UFPR-ALPR-dataset/testing/track0109/track0109[10].png


# Spliting

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)
x_train, x_validation, y_train, y_validation = train_test_split(x_train, y_train, test_size=0.125)

In [7]:
print(y_train.shape, y_test.shape, y_validation.shape)

(3150, 8) (900, 8) (450, 8)


In [8]:
dim_x = 350
dim_y = 350

# Train

In [9]:
train_dir = "../LPD/training/"
if not exists(train_dir):
    makedirs(train_dir)
xy = np.column_stack((x_train, y_train))
x_train = []
y_train = []
for path in tqdm_notebook(xy):
    bb = np.asarray(path[1:], dtype=int)
    im = io.imread(path[0])
    im = im[bb[5]:bb[5]+bb[7], bb[4]:bb[4]+bb[6]]
    dw = 1. / im.shape[1]
    dh = 1. / im.shape[0]
    im = transform.resize(im, (dim_y, dim_x))
    x = bb[1] * dw
    y = bb[0] * dh
    w = bb[3] * dw
    h = bb[2] * dh
    bb = (x, y, w, h)
    y_train.append(bb)
    io.imsave(join(train_dir, path[0].split("/")[-1][:-4]) + ".jpg", im)
    x_train.append(join(train_dir, path[0].split("/")[-1][:-4]) + ".jpg")

HBox(children=(IntProgress(value=0, max=3150), HTML(value='')))

/home/sagacius/.local/lib/python3.5/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/sagacius/.local/lib/python3.5/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


IndexError: too many indices for array

In [10]:
x_train = np.asarray(x_train)
y_train = np.asarray(y_train)
print(x_train.shape, x_train[0])
print(y_train.shape, y_train[0])

(3150,) ../LPD/training/track0112[14].jpg
(3150, 4) [0.44739583 0.64722222 0.04479167 0.02685185]


# Test

In [11]:
test_dir = "../LPD/testing/"
if not exists(test_dir):
    makedirs(test_dir)
xy = np.column_stack((x_test, y_test))
x_test = []
y_test = []
for path in tqdm_notebook(xy):
    im = io.imread(path[0])
    dw = 1. / im.shape[1]
    dh = 1. / im.shape[0]
    im = transform.resize(im, (dim_y, dim_x))
    bb = np.asarray(path[1:], dtype=int)
    x = bb[1] * dw
    y = bb[0] * dh
    w = bb[3] * dw
    h = bb[2] * dh
    bb = (x, y, w, h)
    y_test.append(bb)
    io.imsave(join(test_dir, path[0].split("/")[-1][:-4]) + ".jpg", im)
    x_test.append(join(test_dir, path[0].split("/")[-1][:-4]) + ".jpg")

/home/sagacius/.local/lib/python3.5/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/sagacius/.local/lib/python3.5/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


In [12]:
x_test = np.asarray(x_test)
y_test = np.asarray(y_test)
print(x_test.shape, x_test[0])
print(y_test.shape, y_test[0])

(900,) ../LPD/testing/track0085[22].jpg
(900, 4) [0.19583333 0.675      0.1078125  0.06203704]


# Validation

In [13]:
validation_dir = "../LPD/validation/"
if not exists(validation_dir):
    makedirs(validation_dir)
xy = np.column_stack((x_validation, y_validation))
x_validation = []
y_validation = []
for path in tqdm_notebook(xy):
    im = io.imread(path[0])
    dw = 1. / im.shape[1]
    dh = 1. / im.shape[0]
    im = transform.resize(im, (dim_y, dim_x))
    bb = np.asarray(path[1:], dtype=int)
    x = bb[1] * dw
    y = bb[0] * dh
    w = bb[3] * dw
    h = bb[2] * dh
    bb = (x, y, w, h)
    y_validation.append(bb)
    io.imsave(join(validation_dir, path[0].split("/")[-1][:-4]) + ".jpg", im)
    x_validation.append(join(validation_dir, path[0].split("/")[-1][:-4]) + ".jpg")

/home/sagacius/.local/lib/python3.5/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/sagacius/.local/lib/python3.5/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


In [14]:
x_validation = np.asarray(x_validation)
y_validation = np.asarray(y_validation)
print(x_validation.shape, x_validation[0])
print(y_validation.shape, y_validation[0])

(450,) ../LPD/validation/track0003[23].jpg
(450, 4) [0.44583333 0.42314815 0.04583333 0.02962963]


In [15]:
np.save("../Cache/LPD/new/x_train.npy", x_train)
np.save("../Cache/LPD/new/x_test.npy", x_test)
np.save("../Cache/LPD/new/x_validation.npy", x_validation)
np.save("../Cache/LPD/new/y_train.npy", y_train)
np.save("../Cache/LPD/new/y_test.npy", y_test)
np.save("../Cache/LPD/new/y_validation.npy", y_validation)